# ipyvuetify Tutorial 08 - Custom Components

This is number 8 in a series of ipyvuetify app development tutorials. If you're just getting started with ipyvuetify and haven't checked out the first tutorial "01 Installation and First Steps.ipynb", be sure to check that one out first.

First of all, we'll load the required packages, and test to make sure your environment has all the dependencies set-up successfully:

In [ ]:
import ipyvuetify as v
import ipywidgets as widgets
import traitlets
from warnings import warn


v.Btn(class_='icon ma-2',
      style_='max-width:100px',
      color='success',
      children=[v.Icon(children=['mdi-check'])])


If you see a green button with a checkmark above, you have successfully installed ipyvuetify and enabled the extension. Good work!

If not, refer to the first tutorial and/or the ipyvuetify documentation to set up your system before going further.

## Crawling - A (fairly) Simple Example

`ipyvuetify` provides a template you can use to create your own `vuetify` components.

We will work toward developing an Integer component with custom validation logic.

First we'll give a minimal example to get a feel for the syntax.

In [ ]:
class BasicIntegerInput(v.VuetifyTemplate):
    template = traitlets.Unicode('''
    <template>
      <v-text-field
        label='My Label'
        hint='My Hint'
        type='number'
        placeholder='My Placeholder'
      ></v-text-field>
    </template>
        ''').tag(sync=True)

    def __init__(self,
                 *args,
                 **kwargs):
        super().__init__(*args, **kwargs)



In [ ]:
BasicIntegerInput()

So ! That looks like the `v.TextField` we saw in tutorial 4. 

It's much less useful, though, in this instance; as implemented there is no way to actually get the value once the user has entered it. Nor is there a way to programatically set the value. Next we will show how to implement these improvements.

## Crab Walking - A Slightly Less Simple Example


In [ ]:
class IntegerInput(v.VuetifyTemplate):
    value = traitlets.Unicode().tag(sync=True, allow_null=True)
    template = traitlets.Unicode('''
    <template>
      <v-text-field
        label='My Label'
        hint='My Hint'
        type='number'
        v-model="value"
        placeholder='My Placeholder'
      ></v-text-field>
    </template>
        ''').tag(sync=True)

    def __init__(self,
                 value=None,
                 *args,
                 **kwargs):
        super().__init__(*args, **kwargs)
        if value is not None:
            self.value = str(value)


In [ ]:
test = IntegerInput()
test

In [ ]:
test.value

In [ ]:
test.value='10'

In [ ]:
test.value

In [ ]:
test = IntegerInput(value=10)
test

In [ ]:
test.value

## Running - Integer Input With Limits and Validation

In [ ]:
class IntegerInputWithLimits(v.VuetifyTemplate):
    """
    ipyvuetify integer input with min and max value and appropriate hint

    min_value : int
        Minimum value
    max_value : int
        Maximuim value
    color : str
        ipyvuetify colour
    label : str
        Text label for the input
    value : str
        Value of the input
    filled : bool
        filled style flag
    dense : bool
        dense style flag
    flat : bool
        flat style flag
    rounded : bool
        rounded style flag
    shaped : bool
        shaped style flag
    """
    min_value = traitlets.Integer().tag(sync=True, allow_null=True)
    max_value = traitlets.Integer().tag(sync=True, allow_null=True)
    label = traitlets.Unicode('').tag(sync=True, allow_null=True)
    hint = traitlets.Unicode('').tag(sync=True, allow_null=True)
    value = traitlets.Any().tag(sync=True, allow_null=True)
    color = traitlets.Unicode().tag(sync=True, allow_null=True)
    placeholder = traitlets.Unicode().tag(sync=True, allow_null=True)
    filled = traitlets.Bool().tag(sync=True, allow_null=True)
    dense = traitlets.Bool().tag(sync=True, allow_null=True)
    flat = traitlets.Bool().tag(sync=True, allow_null=True)
    rounded = traitlets.Bool().tag(sync=True, allow_null=True)
    shaped = traitlets.Bool().tag(sync=True, allow_null=True)
    template = traitlets.Unicode('''
    <template>
      <v-text-field
        :label="label"
        :color="color"
        :filled="filled"
        :dense="dense"
        :flat="flat"
        :rounded="rounded"
        :shaped="shaped"
        type="number"
        :hint="hint"
        :placeholder="placeholder"
        v-model="value"
        :rules="[v => !!v || 'Input invalid. Please enter an integer in range ['+String(min_value)+','+String(max_value)+']',
                 v => (v >= min_value) || 'Must be greater than min_value: '+String(min_value),
                 v => (v <= max_value) || 'Must be less than max_value: '+String(max_value)]"
        clearable
      ></v-text-field>
    </template>
        ''').tag(sync=True)

    def __init__(self,
                 *args,
                 min_value=int(-1e20),
                 max_value=int(1e20),
                 color='primary',
                 label='',
                 value=None,
                 filled=False,
                 dense=False,
                 flat=False,
                 rounded=False,
                 shaped=False,
                 **kwargs):
        super().__init__(*args, **kwargs)
        self.label = label
        self.color=color
        self.filled=filled
        self.dense=dense
        self.flat=flat
        self.rounded=rounded
        self.shaped=shaped
        self.min_value=min_value
        self.max_value=max_value
        self.value = value
        self.placeholder = "Enter an integer in range: [{min_value}, {max_value}]".format(
                min_value=self.min_value, max_value=self.max_value)

        try:
            # If value is less than the minimum allowed, set it to the min_value
            if float(self.value)<self.min_value:
                warn('Must be greater than min_value: {min_value}. Setting to {min_value}.'.format(min_value=self.min_value))
                self.value=self.min_value
        
            # If value is greater than the max allowed, set it to the max_value
            if float(self.value)>self.max_value:
                warn('Must be less than max_value: {max_value}. Setting to {max_value}.'.format(max_value=self.max_value))
                self.value=self.max_value
        except TypeError:                  
            self.hint = "Please enter an integer in range: [{min_value}, {max_value}]".format(
                min_value=self.min_value, max_value=self.max_value)

    def __dir__(self):
        # Gives tab completion for class
        return list(self.__dict__['_trait_values'].keys())
            
    @traitlets.observe('value')
    def _observe_value(self, change):
        # When `value` changes, try to change the format of `value` to integer
        #   and if that fails, update the hint and set the value to None
          
        try:
            self.value = int(float(change['new']))
            self.hint = ""
        except:
            self.value=None
            self.hint = "Input Invalid. Please enter an integer in range: [{min_value}, {max_value}]".format(
                min_value=self.min_value, max_value=self.max_value)




In [ ]:
myint = IntegerInputWithLimits(min_value=0,max_value=10)
myint

In [ ]:
myint.value="12-12-12"

In [ ]:
myint.value

In [ ]:
myint.value=None

In [ ]:
myint.value

In [ ]:
# Give it a value that is too small, and it'll give a sensible hint telling you it's too small
myint.value = -10

In [ ]:
# Give it a value that is too big, and it'll give a sensible hint telling you it's too big
myint.value = 20

In [ ]:
# If you set a value within the range, the hint disappears and the value is accepted
myint.value=5

In [ ]:
# If you set the value to be a string, it will convert it to an int (if it can!)
myint.value="10"

## Date Range Dialog

In [ ]:
class DateRangeInputDialog(v.VuetifyTemplate):
    """
    Vuetify Compact, Expandable Daterange Input
    
    Args:
        date_range : list
            A list of two dates [start,end] in format YYYY-mm-dd
    """

    date = traitlets.List([]).tag(sync=True)
    menu = traitlets.Bool(False).tag(sync=True)
    
    template = traitlets.Unicode('''
<template>
   <v-dialog
          ref="menu"
          v-model="menu"
          :close-on-content-click="false"
          :return-value.sync="date"
          transition="scale-transition"
          offset-y
          persistent
          min-width="290px"
          max-width="350px"
        >
      <template v-slot:activator="{ on }">
         <v-card>
            <v-text-field
              v-model="date"
              label="Select Date Range"
              prepend-icon="event"
              readonly
              v-on="on"
            ></v-text-field>
         </template>
         <v-date-picker v-model="date" no-title flat scrollable range>
            <v-spacer></v-spacer>
            <v-btn text color="primary" @click="menu = false">Cancel</v-btn>
            <v-btn text color="primary" @click="$refs.menu.save(date)">OK</v-btn>
         </v-date-picker>
      </v-card>
   </v-dialog>
</template>
        ''').tag(sync=True)
    
    def __init__(self, *args, 
                 date_range=None,menu=False, **kwargs):
        super().__init__(*args, **kwargs)
        self.date = date_range
        self.menu = False
    

In [ ]:
DateRangeInputDialog(date_range=['2020-05-01','2020-05-10'])

## Menu Example

From: https://github.com/mariobuikhuizen/ipyvuetify/blob/master/examples/Examples%20template.ipynb

In [ ]:
class MyMenu(v.VuetifyTemplate):
    
    color = traitlets.Unicode('primary').tag(sync=True)
    items = traitlets.List(['red', 'green', 'purple']).tag(sync=True)
    button_text = traitlets.Unicode('menu').tag(sync=True)
    template = traitlets.Unicode('''
        <v-layout>
            <v-menu offset-y>
                <template v-slot:activator="{ on }">
                    <v-btn
                            :color="color"
                            class="white--text"
                            v-on="on">
                        {{ button_text }}
                    </v-btn>
                </template>
                <v-list>
                    <v-list-item
                            v-for="(item, index) in items"
                            :key="index"
                            @click="menu_click(index)">
                        <v-list-item-title>{{ item }}</v-list-item-title>
                    </v-list-item>
                </v-list>
            </v-menu>
        </v-layout>''').tag(sync=True)
    
    
    def vue_menu_click(self, data):
        self.color = self.items[data]
        self.button_text = self.items[data]
    
    

In [ ]:
test = MyMenu()
test

You can read **or change** the color of the menu with the `color` property, and the button text with the `button_text` property

In [ ]:
test.color

In [ ]:
test.button_text = "My New Button Text"

In [ ]:
test.button_text

## Pandas Data Frame

In [ ]:
import pandas as pd
import traitlets
import ipyvuetify as v
import json

class PandasDataFrame(v.VuetifyTemplate):
    """
    Vuetify DataTable rendering of a pandas DataFrame

    Args:
        data (DataFrame) - the data to render
        title (str) - optional title
        
    Modified from Source: https://jupyter-tutorial.readthedocs.io/de/latest/workspace/jupyter/ipywidgets/libs/ipyvuetify.html
    """

    headers = traitlets.List([]).tag(sync=True, allow_null=True)
    items = traitlets.List([]).tag(sync=True, allow_null=True)
    search = traitlets.Unicode('').tag(sync=True)
    title = traitlets.Unicode('DataFrame').tag(sync=True)
    index_col = traitlets.Unicode('').tag(sync=True)
    template = traitlets.Unicode('''
        <template>
          <v-card>
            <v-card-title>
              <span class="title font-weight-bold">{{ title }}</span>
              <v-spacer></v-spacer>
                <v-text-field
                    v-model="search"
                    append-icon="search"
                    label="Search ..."
                    single-line
                    hide-details
                ></v-text-field>
            </v-card-title>
            <v-data-table
                :headers="headers"
                :items="items"
                :search="search"
                :item-key="index_col"
                :rows-per-page-items="[25, 50, 250, 500]"
            >
                <template v-slot:no-data>
                  <v-alert :value="true" color="error" icon="warning">
                    Sorry, nothing to display here :(
                  </v-alert>
                </template>
                <template v-slot:no-results>
                    <v-alert :value="true" color="warning" icon="warning">
                      Your search for "{{ search }}" found no results.
                    </v-alert>
                </template>
                <template v-slot:items="rows">
                  <td v-for="(element, label, index) in rows.item"
                      @click=cell_click(element)
                      >
                    {{ element }}
                  </td>
                </template>
            </v-data-table>
          </v-card>
        </template>
        ''').tag(sync=True)

    def __init__(self, *args,
                 data=pd.DataFrame(),
                 title=None,
                 **kwargs):
        super().__init__(*args, **kwargs)
        data = data.reset_index()
        self.index_col = data.columns[0]
        headers = [{
              "text": col,
              "value": col
            } for col in data.columns]
        headers[0].update({'align': 'left', 'sortable': True})
        self.headers = headers
        self.items = json.loads(
            data.to_json(orient='records'))
        if title is not None:
            self.title = title


In [ ]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
mytable = PandasDataFrame(data = iris, title='Iris')
mytable

In [ ]:
mytable.search = 'setosa'

In [ ]:
mytable.title = 'Iris Table - New Title!'